In [2]:
import os
import json

import numpy as np
import pandas as pd

import torch
import torchvision

import matplotlib.pyplot as plt
import lightning as L

from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger, CSVLogger
from lightning.pytorch.utilities.model_summary import ModelSummary

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from data_module.data_module import FFTDataModule
from model.AE_model import AECNN1DModel

import optuna
from optuna.integration.pytorch_lightning import PyTorchLightningPruningCallback

In [3]:
TEST = True

random_seed = 42
L.seed_everything(random_seed)

Seed set to 42


42

In [ ]:
n_epochs = 10
patience = 2

optimizer_param_dict = {
    "Adam": (optim.Adam, {
        "lr": 0.001,
    }),
    "SGD": (optim.SGD, {
        "lr": 0.001,
        "momentum": 0.5,
    }),
}
batch_size = 512
optimizer, optimizer_param = optimizer_param_dict["Adam"]
dataset_path = "/nfs/ksdata/tran/HAR_AE/dataset/processed_concat_data"

log_save_dir = "lightning_logs"
log_save_name = "12_AE_train_optuna"

In [ ]:

def objective(trial):
    linear_second_input = trial.suggest_int("linear_second_input", 5, 7)
    print("linear_second_input", linear_second_input)

    trainer = L.Trainer(
        max_epochs=n_epochs,
        enable_checkpointing=False,
        accelerator="auto",
        # check_val_every_n_epoch=10,
        )

    net = AECNN1DModel(
        optimizer = optimizer,
        optimizer_param = optimizer_param, 
        cnn_channel_param = [
            (6, 32, 8, 0, 3),
            (32, 64, 8, 0, 3)
        ],
        linear_channel_param = [
            2**(linear_second_input+2), 2**(linear_second_input+1), 2**linear_second_input
        ],
    )

    model_summary = ModelSummary(net, max_depth=6)
    print("model_summary", model_summary)

    data_module = FFTDataModule(dataset_path=dataset_path, batch_size=batch_size)

    trainer.fit(model=net, datamodule=data_module)

    trainer_test_dict = trainer.logged_metrics
    return trainer_test_dict["val_mse"].item()

In [ ]:
pruner: optuna.pruners.BasePruner = (optuna.pruners.MedianPruner())

study = optuna.create_study(direction="minimize", pruner=pruner)
study.optimize(objective, n_trials=3, timeout=600)

print("Best trial:")
trial = study.best_trial

for key, val in trial.params.items():
    print("{}: {}".format(key, val))

with open("optuna_params.json", "w") as f:
    json.dump(dict(trial.params), f, indent=4)